In [1]:
print("hello")

hello


In [2]:
import os
import math
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

In [17]:
# ============================================================
# 温度3段×3回 ミニ検証セル（1本完結）
#  - generate → raw logits entropy → Aha events → consistency → format → final_reward
# ============================================================

import torch
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------
# config
# ----------------------------
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
DTYPE = torch.bfloat16 if torch.cuda.is_available() else torch.float32

TEMPS = [0.7, 1.0, 1.4]     # boring寄り / 通常 / nonsense増
N_REPEATS = 3              # 各温度で何回生成するか
TOP_P = 0.9
MAX_NEW_TOKENS = 160

PROMPTS = [
    "料理 × 建築で、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。",
    "歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。",
    "新しいAI研究テーマを1つ提案して。タイトルと要旨(5行)で。",
]

# ----------------------------
# load model
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

# ----------------------------
# utilities
# ----------------------------
def entropy_from_logits(logits):
    p = torch.softmax(logits.float(), dim=-1)
    p = torch.clamp(p, min=1e-12)
    return float(-(p * torch.log(p)).sum().item())

def cosine_similarity(a, b):
    a = a.float()
    b = b.float()
    denom = (a.norm() * b.norm()).clamp(min=1e-12)
    return float((a @ b / denom).item())

def smooth(x, win=7):
    if win <= 1:
        return np.array(x, dtype=np.float32)
    k = np.ones(win, dtype=np.float32) / win
    return np.convolve(np.array(x, dtype=np.float32), k, mode="same")

def aha_events(entropies, K=32, peak_thr=2.0, drop_thr=0.6, smooth_win=7):
    H = smooth(entropies, win=smooth_win)
    T = len(H)
    events = []
    t = 1
    while t < T - 1:
        if H[t] > H[t-1] and H[t] > H[t+1] and H[t] >= peak_thr:
            r = min(T, t + K)
            t_min = t + int(np.argmin(H[t:r]))
            drop = float(H[t] - H[t_min])
            if drop >= drop_thr:
                events.append({
                    "t_peak": int(t),
                    "t_drop": int(t_min),
                    "H_peak": float(H[t]),
                    "H_drop": float(H[t_min]),
                    "drop": drop,
                })
                t = t_min + 1
                continue
        t += 1
    score = float(sum(e["drop"] for e in events))
    return events, score

def compute_consistency(hidden_vecs, W=8):
    cons = []
    T = len(hidden_vecs)
    for t in range(T):
        if t < W or t + W >= T:
            cons.append(0.0)
            continue
        m_prev = torch.stack(hidden_vecs[t-W:t]).mean(dim=0)
        m_cur  = torch.stack(hidden_vecs[t:t+W]).mean(dim=0)
        cons.append(cosine_similarity(m_prev, m_cur))
    return cons

def consistency_drop(hidden_vecs):
    sims = [cosine_similarity(hidden_vecs[t-1], hidden_vecs[t]) for t in range(1, len(hidden_vecs))]
    return float(min(sims)) if sims else 0.0

def format_score_strict(text: str) -> float:
    has_title = bool(re.search(r"(題名|タイトル|Title)\s*[:：]", text))
    has_abs   = bool(re.search(r"(要旨|Abstract)\s*[:：]", text))

    abs_part = ""
    m = re.search(r"(要旨|Abstract)\s*[:：](.*)", text, flags=re.S)
    if m:
        abs_part = m.group(2).strip()

    abs_lines = [ln.strip() for ln in abs_part.splitlines() if ln.strip()]
    five_lines = (len(abs_lines) >= 5)

    score = 0.0
    score += 0.34 if has_title else 0.0
    score += 0.33 if has_abs else 0.0
    score += 0.33 if five_lines else 0.0
    return float(score)

def final_reward(aha_score, cons_mean_valid, cons_drop, fmt_score):
    # cons_drop が負なら0にして “仮説急変” を即死させる
    return float(aha_score) * max(0.0, float(cons_mean_valid)) * max(0.0, float(cons_drop)) * (float(fmt_score) ** 2)

# ----------------------------
# generate + raw trace
# ----------------------------
@torch.no_grad()
def generate_and_trace(prompt, max_new_tokens=160, temperature=1.0, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    p_len = inputs["input_ids"].shape[1]

    # generate (token ids)
    seq = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        return_dict_in_generate=False,
    )[0]  # (p_len + T,)

    # 生成部分（token ids）
    gen_part = seq[p_len:]
    gen_text = tokenizer.decode(gen_part, skip_special_tokens=True)

    # ★文字列連結しない：token ids をそのまま forward
    seq2 = seq.unsqueeze(0)  # (1, seq_len)
    attn = torch.ones_like(seq2, device=seq2.device)

    out = model(input_ids=seq2, attention_mask=attn, output_hidden_states=True, use_cache=False)
    logits = out.logits[0]            # (seq_len, vocab)
    hidden = out.hidden_states[-1][0] # (seq_len, hidden_dim)

    T = gen_part.shape[0]
    entropies = []
    hidden_vecs = []

    # 位置 idx の logits は token[idx+1] を予測
    # 生成トークン t は seq[p_len + t] なので、対応する logits は idx = (p_len + t - 1)
    for t in range(T):
        idx = p_len + t - 1
        if idx < 0 or idx >= logits.shape[0]:
            break
        entropies.append(entropy_from_logits(logits[idx]))
        hidden_vecs.append(hidden[idx].detach().cpu())

    return gen_text, entropies, hidden_vecs

# ----------------------------
# run
# ----------------------------
rows = []
run_id = 0

for p_i, prompt in enumerate(PROMPTS):
    for temp in TEMPS:
        for rep in range(N_REPEATS):
            run_id += 1
            text, ent, hidden_vecs = generate_and_trace(
                prompt,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=temp,
                top_p=TOP_P,
            )

            events, aha_score = aha_events(ent, K=32, peak_thr=2.0, drop_thr=0.6, smooth_win=7)
            cons = compute_consistency(hidden_vecs, W=8)
            cons_valid = cons[8:-8] if len(cons) > 16 else cons
            cons_mean_valid = float(np.mean(cons_valid)) if len(cons_valid) else 0.0
            cons_drop = consistency_drop(hidden_vecs)
            fmt = format_score_strict(text)
            R = final_reward(aha_score, cons_mean_valid, cons_drop, fmt)

            rows.append({
                "run_id": run_id,
                "prompt_id": p_i,
                "temperature": temp,
                "repeat": rep,
                "prompt": prompt,
                "output_head": text[:120].replace("\n", "\\n") + "...",
                "T_steps": len(ent),
                "entropy_mean": float(np.mean(ent)),
                "entropy_max": float(np.max(ent)),
                "aha_event_count": len(events),
                "aha_event_score": float(aha_score),
                "aha_event_maxdrop": float(max([e["drop"] for e in events], default=0.0)),
                "cons_mean_valid": cons_mean_valid,
                "consistency_drop": float(cons_drop),
                "format_score": float(fmt),
                "final_reward": float(R),
                "note": f"events(head)={events[:2]}",
                "label_manual": "",  # creative / boring / nonsense を後で手で入れる
            })

df = pd.DataFrame(rows)

# まず一覧（final_reward順）
display(df.sort_values("final_reward", ascending=False).reset_index(drop=True))

# 温度ごとの集計（平均・中央値）
summary = df.groupby("temperature").agg(
    n=("final_reward", "count"),
    final_mean=("final_reward", "mean"),
    final_median=("final_reward", "median"),
    aha_mean=("aha_event_score", "mean"),
    cons_mean=("cons_mean_valid", "mean"),
    drop_mean=("consistency_drop", "mean"),
    fmt_mean=("format_score", "mean"),
).reset_index()

display(summary)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

,run_id,prompt_id,temperature,repeat,prompt,output_head,T_steps,entropy_mean,entropy_max,aha_event_count,aha_event_score,aha_event_maxdrop,cons_mean_valid,consistency_drop,format_score,final_reward,note,label_manual
0,27,2,1.4,2,新しいAI研究テーマを1つ提案して。タイトルと要旨(5行)で。,タイトルは「未来のAI研究」を含めて。\n\nタイトル: 未来のAI研究 - 高度認知能力向...,160,1.251774,6.235067,3,6.609722,3.303767,0.884576,0.084023,0.67,0.220528,"events(head)=[{'t_peak': 3, 't_drop': 15, 'H_p...",
1,14,1,1.0,1,歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。,以下のような観点から歴史とサイバネティクスを組み合わせた研究テーマを提案します。\n\nタ...,160,1.127891,4.803109,3,5.306203,2.337435,0.885927,0.082182,0.67,0.173424,"events(head)=[{'t_peak': 3, 't_drop': 11, 'H_p...",
2,9,0,1.4,2,料理 × 建築で、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。,"研究テーマ: ""食を通じた建築体験の開発""\n\nタイトル:\n「味覚と空間の融合 – 通...",160,1.493148,5.589054,6,7.751659,1.774551,0.901896,0.028996,0.67,0.090999,"events(head)=[{'t_peak': 27, 't_drop': 49, 'H_...",
3,13,1,1.0,0,歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。,- 質問が追加されました\n歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。...,160,1.038657,5.381445,2,4.648714,2.627976,0.887400,0.021904,0.67,0.040562,"events(head)=[{'t_peak': 3, 't_drop': 18, 'H_p...",
4,23,2,1.0,1,新しいAI研究テーマを1つ提案して。タイトルと要旨(5行)で。,タイトル：音声合成の多言語適用と感情表現の向上\n\n要旨：本研究では、音声合成技術を多言...,160,0.979979,4.527391,1,2.036939,2.036939,0.879061,0.037295,0.67,0.029978,"events(head)=[{'t_peak': 7, 't_drop': 22, 'H_p...",
5,12,1,0.7,2,歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。,例えば、「歴史的サイバネティクス」の研究テーマは「伝統的な情報処理手法と現代のサイバネティ...,160,1.057639,4.842188,1,2.281402,2.281402,0.890802,0.030966,0.67,0.028250,"events(head)=[{'t_peak': 4, 't_drop': 32, 'H_p...",
6,22,2,1.0,0,新しいAI研究テーマを1つ提案して。タイトルと要旨(5行)で。,タイトル: マルチモーダル・クロスディシプリンラリアルティの統合によるAI創造性の向上\n...,160,0.921155,4.190171,1,1.875497,1.875497,0.877999,0.031973,0.67,0.023635,"events(head)=[{'t_peak': 16, 't_drop': 36, 'H_...",
7,21,2,0.7,2,新しいAI研究テーマを1つ提案して。タイトルと要旨(5行)で。,題名: 機械学習による個別化医療の実現\n\n要旨: この研究では、患者の個人情報や病歴か...,160,0.982274,5.261212,1,1.917202,1.917202,0.857356,0.023509,0.67,0.017347,"events(head)=[{'t_peak': 14, 't_drop': 23, 'H_...",
8,18,1,1.4,2,歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。タイトルと要旨(5行)で。,「歴史的サイバネティクス：情報社会における過去と現在の関連性探求」\n要旨: 近年、情報社...,160,1.061070,4.811566,1,1.372735,1.372735,0.895809,0.083346,0.33,0.011161,"events(head)=[{'t_peak': 14, 't_drop': 30, 'H_...",
9,24,2,1.0,2,新しいAI研究テーマを1つ提案して。タイトルと要旨(5行)で。,タイトル: 人間中心の倫理的AI開発\n\n要旨: 本研究では、AIの開発プロセスにおいて...,160,0.955855,3.970444,1,1.819769,1.819769,0.878471,0.010557,0.67,0.007576,"events(head)=[{'t_peak': 142, 't_drop': 159, '...",


,temperature,n,final_mean,final_median,aha_mean,cons_mean,drop_mean,fmt_mean
0,0.7,9,0.005066,0.000000,4.347893,0.844415,-0.008091,0.446667
1,1.0,9,0.030575,0.007576,5.794154,0.892238,0.007990,0.484444
2,1.4,9,0.036488,0.000000,5.272402,0.879943,0.021184,0.372222


In [18]:
# ============================================================
# 温度3段×3回 ミニ検証セル（1本完結・改良版）
#  - generate → raw logits entropy → Aha events → consistency(q) → format gate → final_reward_v2
# ============================================================

import torch
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

# ----------------------------
# config
# ----------------------------
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
DTYPE = torch.bfloat16 if torch.cuda.is_available() else torch.float32

TEMPS = [0.7, 1.0, 1.4]     # boring寄り / 通常 / nonsense増
N_REPEATS = 3              # 各温度で何回生成するか
TOP_P = 0.9
MAX_NEW_TOKENS = 180       # 少し余裕（必要なら160に戻してOK）

# ★プロンプトを増やす（RLで試す“考える系”を混ぜる）
#  - 形式固定 + 禁止事項で「OKWAVE/サイト断片」誘発を抑える
PROMPTS = [
    # 1) 異分野融合（creative）
    """あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテゴリ表示。
タイトル：<1行>
要旨：
- <1行>
- <1行>
- <1行>
- <1行>
- <1行>
料理 × 建築で、新しい研究テーマを1つ提案して。""",

    """あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテゴリ表示。
タイトル：<1行>
要旨：
- <1行>
- <1行>
- <1行>
- <1行>
- <1行>
歴史 × サイバネティクスで、新しい研究テーマを1つ提案して。""",

    # 2) 反例・落とし穴（論理性）
    """あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテゴリ表示。
タイトル：<1行>
要旨：
- まず主張（1行）
- その主張が破綻する反例（1行）
- 反例の原因分析（1行）
- 改善案（1行）
- 検証方法（1行）
「LLMの創造性をエントロピーだけで測れる」という主張に対して、反例を作って分析して。""",

    # 3) 制約付き設計（planning）
    """あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテゴリ表示。
タイトル：<1行>
要旨：
- 目的（1行）
- 制約（1行）
- 方法（1行）
- 失敗しやすい点（1行）
- 評価指標（1行）
GPUメモリが48GBで、PPO+LoRAを回すための実装設計を提案して（具体的に）。""",

    # 4) アルゴリズム案（構造化）
    """あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテゴリ表示。
タイトル：<1行>
要旨：
- 問題設定（1行）
- アルゴリズムの核心（1行）
- 擬似コードの要点（1行）
- 計算量/ボトルネック（1行）
- 失敗ケース（1行）
「Ahaイベント検出」を改良して、nonsenseをさらに落とすアルゴリズム案を提案して。""",

    # 5) 通常の研究テーマ（boring寄り）
    """あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテゴリ表示。
タイトル：<1行>
要旨：
- <1行>
- <1行>
- <1行>
- <1行>
- <1行>
新しいAI研究テーマを1つ提案して。""",
]

# ----------------------------
# load model
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

# ----------------------------
# utilities
# ----------------------------
def entropy_from_logits(logits):
    p = torch.softmax(logits.float(), dim=-1)
    p = torch.clamp(p, min=1e-12)
    return float(-(p * torch.log(p)).sum().item())

def cosine_similarity(a, b):
    a = a.float()
    b = b.float()
    denom = (a.norm() * b.norm()).clamp(min=1e-12)
    return float((a @ b / denom).item())

def smooth(x, win=7):
    if win <= 1:
        return np.array(x, dtype=np.float32)
    k = np.ones(win, dtype=np.float32) / win
    return np.convolve(np.array(x, dtype=np.float32), k, mode="same")

def aha_events(entropies, K=32, peak_thr=2.0, drop_thr=0.6, smooth_win=7):
    H = smooth(entropies, win=smooth_win)
    T = len(H)
    events = []
    t = 1
    while t < T - 1:
        if H[t] > H[t-1] and H[t] > H[t+1] and H[t] >= peak_thr:
            r = min(T, t + K)
            t_min = t + int(np.argmin(H[t:r]))
            drop = float(H[t] - H[t_min])
            if drop >= drop_thr:
                events.append({
                    "t_peak": int(t),
                    "t_drop": int(t_min),
                    "H_peak": float(H[t]),
                    "H_drop": float(H[t_min]),
                    "drop": drop,
                })
                t = t_min + 1
                continue
        t += 1
    score = float(sum(e["drop"] for e in events))
    return events, score

def compute_consistency(hidden_vecs, W=8):
    cons = []
    T = len(hidden_vecs)
    for t in range(T):
        if t < W or t + W >= T:
            cons.append(0.0)
            continue
        m_prev = torch.stack(hidden_vecs[t-W:t]).mean(dim=0)
        m_cur  = torch.stack(hidden_vecs[t:t+W]).mean(dim=0)
        cons.append(cosine_similarity(m_prev, m_cur))
    return cons

# ★minではなく下位分位（頑丈）
def consistency_drop_q(hidden_vecs, q=0.05, ignore_first=8, ignore_last=8):
    sims = []
    T = len(hidden_vecs)
    for t in range(1, T):
        if t < ignore_first or t > T - ignore_last:
            continue
        sims.append(cosine_similarity(hidden_vecs[t-1], hidden_vecs[t]))
    if not sims:
        return 0.0
    return float(np.quantile(np.array(sims, dtype=np.float32), q))

def format_score_strict(text: str) -> float:
    has_title = bool(re.search(r"(題名|タイトル|Title)\s*[:：]", text))
    has_abs   = bool(re.search(r"(要旨|Abstract)\s*[:：]", text))

    abs_part = ""
    m = re.search(r"(要旨|Abstract)\s*[:：](.*)", text, flags=re.S)
    if m:
        abs_part = m.group(2).strip()

    abs_lines = [ln.strip() for ln in abs_part.splitlines() if ln.strip()]
    five_lines = (len(abs_lines) >= 5)

    score = 0.0
    score += 0.34 if has_title else 0.0
    score += 0.33 if has_abs else 0.0
    score += 0.33 if five_lines else 0.0
    return float(score)

# ★0だけ落とすゲート
def format_gate(text: str) -> float:
    return 0.0 if format_score_strict(text) <= 0.0 else 1.0

# ★final_rewardをv2に
def final_reward_v2(aha_score, cons_mean_valid, cons_drop_q, fmt_gate):
    if fmt_gate <= 0.0:
        return 0.0
    # drop_q が少し負でも即死させずに0〜1へ
    drop = (float(cons_drop_q) + 0.1) / 1.1
    drop = float(np.clip(drop, 0.0, 1.0))
    return float(aha_score) * max(0.0, float(cons_mean_valid)) * drop

# ----------------------------
# generate + raw trace
# ----------------------------
@torch.no_grad()
def generate_and_trace(prompt, max_new_tokens=160, temperature=1.0, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    p_len = inputs["input_ids"].shape[1]

    seq = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        return_dict_in_generate=False,
    )[0]

    gen_part = seq[p_len:]
    gen_text = tokenizer.decode(gen_part, skip_special_tokens=True)

    seq2 = seq.unsqueeze(0)
    attn = torch.ones_like(seq2, device=seq2.device)

    out = model(input_ids=seq2, attention_mask=attn, output_hidden_states=True, use_cache=False)
    logits = out.logits[0]
    hidden = out.hidden_states[-1][0]

    T = gen_part.shape[0]
    entropies = []
    hidden_vecs = []

    for t in range(T):
        idx = p_len + t - 1
        if idx < 0 or idx >= logits.shape[0]:
            break
        entropies.append(entropy_from_logits(logits[idx]))
        hidden_vecs.append(hidden[idx].detach().cpu())

    return gen_text, entropies, hidden_vecs

# ----------------------------
# run
# ----------------------------
rows = []
run_id = 0

for p_i, prompt in enumerate(PROMPTS):
    for temp in TEMPS:
        for rep in range(N_REPEATS):
            run_id += 1
            text, ent, hidden_vecs = generate_and_trace(
                prompt,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=temp,
                top_p=TOP_P,
            )

            events, aha_score = aha_events(ent, K=32, peak_thr=2.0, drop_thr=0.6, smooth_win=7)
            cons = compute_consistency(hidden_vecs, W=8)
            cons_valid = cons[8:-8] if len(cons) > 16 else cons
            cons_mean_valid = float(np.mean(cons_valid)) if len(cons_valid) else 0.0

            cons_drop = consistency_drop_q(hidden_vecs, q=0.05, ignore_first=8, ignore_last=8)

            fmt_s = format_score_strict(text)
            fmt_g = format_gate(text)

            R = final_reward_v2(aha_score, cons_mean_valid, cons_drop, fmt_g)

            rows.append({
                "run_id": run_id,
                "prompt_id": p_i,
                "temperature": temp,
                "repeat": rep,
                "prompt": prompt[:80].replace("\n", " ") + "...",
                "output_head": text[:140].replace("\n", "\\n") + "...",
                "T_steps": len(ent),
                "entropy_mean": float(np.mean(ent)) if len(ent) else 0.0,
                "entropy_max": float(np.max(ent)) if len(ent) else 0.0,
                "aha_event_count": len(events),
                "aha_event_score": float(aha_score),
                "aha_event_maxdrop": float(max([e["drop"] for e in events], default=0.0)),
                "cons_mean_valid": cons_mean_valid,
                "cons_drop_q05": float(cons_drop),
                "format_score_strict": float(fmt_s),
                "format_gate": float(fmt_g),
                "final_reward": float(R),
                "note": f"events(head)={events[:2]}",
                "label_manual": "",
            })

df = pd.DataFrame(rows)

display(df.sort_values("final_reward", ascending=False).reset_index(drop=True))

summary = df.groupby("temperature").agg(
    n=("final_reward", "count"),
    final_mean=("final_reward", "mean"),
    final_median=("final_reward", "median"),
    aha_mean=("aha_event_score", "mean"),
    cons_mean=("cons_mean_valid", "mean"),
    dropq_mean=("cons_drop_q05", "mean"),
    fmt_strict_mean=("format_score_strict", "mean"),
    fmt_gate_rate=("format_gate", "mean"),
).reset_index()

display(summary)

/app/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

,run_id,prompt_id,temperature,repeat,prompt,output_head,T_steps,entropy_mean,entropy_max,aha_event_count,aha_event_score,aha_event_maxdrop,cons_mean_valid,cons_drop_q05,format_score_strict,format_gate,final_reward,note,label_manual
0,48,5,0.7,2,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,タイトルは「人工知能の倫理的影響と社会的課題」です。\n\nタイトル：人工知能の倫理的影響と...,180,0.935474,4.628728,2,4.777727,2.414448,0.837802,0.240800,1.00,1.0,1.240135,"events(head)=[{'t_peak': 3, 't_drop': 30, 'H_p...",
1,27,2,1.4,2,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,\n\n\nタイトル：LLMの創造性をエントロピーだけで測れるとする主張の破綻\n要旨：LL...,180,1.018829,5.140790,3,5.559655,2.521609,0.861348,0.178591,0.67,1.0,1.212831,"events(head)=[{'t_peak': 104, 't_drop': 115, '...",
2,16,1,1.4,0,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,具体的な内容について3行程度述べてください。 标题：サイバネティックスと歴史の融合：過去の行...,180,1.115999,4.633882,3,5.499461,1.978748,0.875362,0.163411,0.33,1.0,1.152788,"events(head)=[{'t_peak': 48, 't_drop': 56, 'H_...",
3,4,0,1.0,0,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,ただし、その理由も含めて。\nタイトル：料理と建築の融合：空間感覚と味覚の相関関係探求\n要...,180,1.241612,4.586817,4,5.658608,1.767261,0.881629,0.146162,0.67,1.0,1.116409,"events(head)=[{'t_peak': 3, 't_drop': 31, 'H_p...",
4,17,1,1.4,1,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,その理由を含めて説明。\nタイトル：サイバネティックスに基づく歴史情報の整理と利用\n要旨：...,180,1.117653,4.337606,3,5.081525,1.747712,0.863492,0.176281,0.67,1.0,1.102074,"events(head)=[{'t_peak': 23, 't_drop': 30, 'H_...",
5,35,3,1.4,1,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,タイトル：48GB GPUメモリを使用したPPO+LoRAの実装設計\n\n要旨：\n- ...,180,0.784560,4.791725,2,4.931791,2.549680,0.804436,0.185345,1.00,1.0,1.029140,"events(head)=[{'t_peak': 101, 't_drop': 123, '...",
6,15,1,1.0,2,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,具体的な手法や実施方法については触れず、「どのような観点から研究を行うべきか」について記述し...,180,1.108339,5.021090,3,4.592279,1.737290,0.864201,0.177944,0.67,1.0,1.002784,"events(head)=[{'t_peak': 5, 't_drop': 30, 'H_p...",
7,38,4,0.7,1,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,具体的な手法や詳細な説明は省きますが、その改良点についての要約のみを記述してください。\nタ...,180,1.096291,4.635312,2,4.439522,2.688587,0.881703,0.168802,1.00,1.0,0.956528,"events(head)=[{'t_peak': 3, 't_drop': 34, 'H_p...",
8,5,0,1.0,1,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,タイトルと要旨を記載せよ。\nタイトル：食の空間設計：料理と建築の融合による新たな居住空間の...,180,1.244245,5.148126,3,5.017167,1.904463,0.876697,0.137015,1.00,1.0,0.947743,"events(head)=[{'t_peak': 15, 't_drop': 32, 'H_...",
9,8,0,1.4,1,あなたは研究者です。次の形式だけで出力してください。禁止：URL/サイト名/引用/日付/カテ...,<1行>\n\nタイトル：料理と建築の整合性を探索する：空間構成による味覚体験の改善\n\...,180,1.192970,4.418414,3,4.852893,2.045201,0.882389,0.139851,1.00,1.0,0.933703,"events(head)=[{'t_peak': 15, 't_drop': 31, 'H_...",


,temperature,n,final_mean,final_median,aha_mean,cons_mean,dropq_mean,fmt_strict_mean,fmt_gate_rate
0,0.7,18,0.395891,0.411509,1.938331,0.856811,0.165220,0.926667,1.000000
1,1.0,18,0.410890,0.419996,2.217307,0.863961,0.168380,0.852222,0.944444
2,1.4,18,0.579477,0.625315,3.043878,0.853040,0.175809,0.722222,0.888889
